In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
import os
import scipy.signal
from numba import jit, prange

from sklearn.model_selection import train_test_split
from joblib import Parallel, delayed
from sklearn.metrics import classification_report
from tqdm import tqdm
from vmdpy import VMD
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

os.chdir('../')
os.chdir('data/')

## Functions needed for execution

format_text() takes the dataframe and the column index which contains the text that needs to be cleaned, in this case the cleaning process was focussed for Tweets which include removal of special charactors, links and numbers.

In [2]:
def format_text(df,col):
      #Remove @ tags
      comp_df = df.copy()

      # remove all the punctuation
      comp_df[col] = comp_df[col].str.replace(r'(@\w*)','')

      #Remove URL
      comp_df[col] = comp_df[col].str.replace(r"http\S+", "")

      #Remove # tag 
      comp_df[col] = comp_df[col].str.replace('#',"")

      #Remove all non-character
      comp_df[col] = comp_df[col].str.replace(r"[^a-zA-Z ]","")

      # Remove extra space
      comp_df[col] = comp_df[col].str.replace(r'( +)'," ")
      comp_df[col] = comp_df[col].str.strip()

      # Change to lowercase
      comp_df[col] = comp_df[col].str.lower()
      comp_df[col] = comp_df[col].str.replace('httpurl', '')
      return comp_df

From the given modes, using the SciPy package the mode containing the maximum energy will be selected.

In [3]:
@jit(parallel=True,forceobj=True)
def energy(u):
# Estimate PSD `S_xx_welch` at discrete frequencies `f_welch`
    f_welch, S_xx_welch = scipy.signal.welch(u)
    # Integrate PSD over spectral bandwidth
    # to obtain signal power `P_welch`
    df_welch = f_welch[1] - f_welch[0]
    return np.sum(S_xx_welch) * df_welch

In [4]:
@jit(parallel=True,forceobj=True)
def maxvdm(f):
    alpha = 3     
    tau = 0            
    K = 2       
    DC = 0             
    init = 1           
    tol = 1e-8
    u, u_hat, omega = VMD(f, alpha, tau, K, DC, init, tol) 
    energy_array=[]
    for i in u:
        energy_array.append(energy(i))
    ind = np.argmax(energy_array)
    return u[ind]

@jit(parallel=True,forceobj=True)
def extract(features):
  X = []
  for i in features:
    X.append(maxvdm(i))
  return X

## Training Data:

Loading and Pre-Processing the Tweets Data

In [25]:
train = pd.read_csv('train.tsv',sep='\t')

In [27]:
train = format_text(train,'Text')
X = train['Text'].tolist()
Y_train = train['Label']

Converting String Labels into Numeric Values with LabelEncoder

In [28]:
le = LabelEncoder()
le.fit(Y_train)
Y_train = le.transform(Y_train)

Computing the TF-IDF vectors from the given corpus of training data

In [29]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(X).toarray()

From the functions section, the maxvdm() which extracts K modes and returns the mode with the highest energy is called on every line.

In [ ]:
X_data = [maxvdm(i) for i in tqdm(features)]

In [31]:
df = pd.DataFrame(X_data)
df['l'] = Y_train

## Testing Data:

The process of testing is similar to the above-mentioned training method, the same object tfidf and labelencoder must be called.

In [32]:
test = pd.read_csv('/content/test.tsv',sep='\t',header=None)
test = format_text(test,1)
X_test = test[1].tolist()
Y_test = le.transform(test[2])

In [ ]:
features_test = tfidf.transform(X_test).toarray()
X_test = [maxvdm(i) for i in tqdm(features_test)]

## Evaluating Model Performance:

In [37]:
lr = LogisticRegression(random_state=0)
lr.fit(X_data,Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [38]:
y_pred = lr.predict(X_test)

In [39]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.56      0.64       944
           1       0.68      0.84      0.75      1056

    accuracy                           0.71      2000
   macro avg       0.72      0.70      0.70      2000
weighted avg       0.72      0.71      0.70      2000

